### Setup

* h5py - http://www.h5py.org
    * `$ pip install h5py`
* keras - https://keras.io
    * `$ pip install keras`
* TensorFlow - https://www.tensorflow.org
    * `$ pip install tensorflow`
* TensorFrames - https://spark-packages.org/package/databricks/tensorframes 
    * ```$ cd ~/spark/jars 
 $ wget http://dl.bintray.com/spark-packages/maven/databricks/tensorframes/0.8.1-s_2.11/tensorframes-0.8.1-s_2.11.jar```
 
* Spark Deep Learning - https://spark-packages.org/package/databricks/spark-deep-learning
* Flowers dataset - https://www.tensorflow.org/tutorials/image_retraining


In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.types import *

spark = SparkSession\
    .builder\
    .appName("chapter-31-Deep-Learning")\
    .getOrCreate()

import os
SPARK_BOOK_DATA_PATH = os.environ['SPARK_BOOK_DATA_PATH']

In [2]:
## from sparkdl import readImages
## not working

In [3]:
img_dir = SPARK_BOOK_DATA_PATH + '/data/deep-learning-images/'
## image_df = readImages(img_dir)

## workaround: https://stackoverflow.com/questions/49643603/spark-deep-learning-import-error
from pyspark.ml.image import ImageSchema

image_df = ImageSchema.readImages(img_dir)

In [4]:
# COMMAND ----------

image_df.printSchema()

root
 |-- image: struct (nullable = true)
 |    |-- origin: string (nullable = true)
 |    |-- height: integer (nullable = false)
 |    |-- width: integer (nullable = false)
 |    |-- nChannels: integer (nullable = false)
 |    |-- mode: integer (nullable = false)
 |    |-- data: binary (nullable = false)



In [5]:
image_df.count()

1

In [7]:
image_df.show(truncate=False)

+----------------------------------------------------------------------------------------------------------------+
|image                                                                                                           |
+----------------------------------------------------------------------------------------------------------------+
|[file:/home/gong/spark/books/Spark-The-Definitive-Guide/data/deep-learning-images/LICENSE.txt, -1, -1, -1, -1, ]|
+----------------------------------------------------------------------------------------------------------------+



In [8]:
# COMMAND ----------

# from sparkdl import readImages
from pyspark.sql.functions import lit
tulips_df = ImageSchema.readImages(img_dir + "/tulips").withColumn("label", lit(1))

In [ ]:
tulips_df.count()

In [11]:
## DONT RUN this

## tulips_df.show(5,False)


#### ERROR

```
IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)
```

#### FIX (not working)
https://stackoverflow.com/questions/43490495/how-to-set-notebookapp-iopub-data-rate-limit-and-others-notebookapp-settings-in

In [12]:
daisy_df = ImageSchema.readImages(img_dir + "/daisy").withColumn("label", lit(0))

In [13]:
daisy_df.count()

Py4JJavaError: An error occurred while calling o65.count.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 5.0 failed 1 times, most recent failure: Lost task 0.0 in stage 5.0 (TID 5, localhost, executor driver): java.lang.IllegalArgumentException: Numbers of source Raster bands and source color space components do not match
	at java.awt.image.ColorConvertOp.filter(ColorConvertOp.java:482)
	at com.sun.imageio.plugins.jpeg.JPEGImageReader.acceptPixels(JPEGImageReader.java:1288)
	at com.sun.imageio.plugins.jpeg.JPEGImageReader.readImage(Native Method)
	at com.sun.imageio.plugins.jpeg.JPEGImageReader.readInternal(JPEGImageReader.java:1255)
	at com.sun.imageio.plugins.jpeg.JPEGImageReader.read(JPEGImageReader.java:1058)
	at javax.imageio.ImageIO.read(ImageIO.java:1462)
	at javax.imageio.ImageIO.read(ImageIO.java:1357)
	at org.apache.spark.ml.image.ImageSchema$.decode(ImageSchema.scala:139)
	at org.apache.spark.ml.image.ImageSchema$$anonfun$readImages$2$$a

In [ ]:
tulips_train, tulips_test = tulips_df.randomSplit([0.6, 0.4])

daisy_train, daisy_test = daisy_df.randomSplit([0.6, 0.4])

train_df = tulips_train.unionAll(daisy_train)
test_df = tulips_test.unionAll(daisy_test)


# COMMAND ----------

from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline
from sparkdl import DeepImageFeaturizer
featurizer = DeepImageFeaturizer(inputCol="image", outputCol="features",
  modelName="InceptionV3")
lr = LogisticRegression(maxIter=1, regParam=0.05, elasticNetParam=0.3,
  labelCol="label")
p = Pipeline(stages=[featurizer, lr])
p_model = p.fit(train_df)


# COMMAND ----------

from pyspark.ml.evaluation import MulticlassClassificationEvaluator
tested_df = p_model.transform(test_df)
evaluator = MulticlassClassificationEvaluator(metricName="accuracy")
print("Test set accuracy = " + str(evaluator.evaluate(tested_df.select(
  "prediction", "label"))))


# COMMAND ----------

from pyspark.sql.types import DoubleType
from pyspark.sql.functions import expr
# a simple UDF to convert the value to a double
def _p1(v):
  return float(v.array[1])
p1 = udf(_p1, DoubleType())
df = tested_df.withColumn("p_1", p1(tested_df.probability))
wrong_df = df.orderBy(expr("abs(p_1 - label)"), ascending=False)
wrong_df.select("filePath", "p_1", "label").limit(10).show()


# COMMAND ----------

from sparkdl import readImages, DeepImagePredictor
image_df = readImages(img_dir)
predictor = DeepImagePredictor(
  inputCol="image",
  outputCol="predicted_labels",
  modelName="InceptionV3",
  decodePredictions=True,
  topK=10)
predictions_df = predictor.transform(image_df)


# COMMAND ----------

df = p_model.transform(image_df)


# COMMAND ----------

from keras.applications import InceptionV3
from sparkdl.udf.keras_image_model import registerKerasImageUDF
from keras.applications import InceptionV3
registerKerasImageUDF("my_keras_inception_udf", InceptionV3(weights="imagenet"))


# COMMAND ----------